In [ ]:
%pip install imblearn shap

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import random

In [ ]:
# Define bucket and file names
bucket_name = "sagemaker-capstone-bucket"
subset_file = "train/second_model_subset.csv"
feature_file = "train/second_features.csv"

# Initialize S3 client
s3_client = boto3.client("s3")

In [ ]:
# Read subset CSV
subset_obj = s3_client.get_object(Bucket=bucket_name, Key=subset_file)
subset = pd.read_csv(StringIO(subset_obj["Body"].read().decode("utf-8")))

In [ ]:
pneumonia_obj = s3_client.get_object(Bucket=bucket_name, Key=feature_file)
pneumonia = pd.read_csv(StringIO(pneumonia_obj["Body"].read().decode("utf-8")))

In [ ]:
pneumonia.head()

In [ ]:
pneu = pneumonia[pneumonia['subject_id'].notna()]

pneu = pneu[[
     'subject_id',
     'charttime',
     'hadm_id',
     'stay_id',
     'heart_rate',
     'sbp',
     'sbp_ni',
     'mbp',
     'mbp_ni',
     'resp_rate',
     'temperature',
     'platelet',
     'wbc',
     'bands',
     'lactate',
     'inr',
     'ptt',
     'creatinine',
     'bilirubin',
     'pneumonia'
]]

# getting most recent lab result
recents = (pneu.sort_values(['subject_id', 'hadm_id', 'stay_id', 'charttime'])
           .groupby(['subject_id', 'hadm_id', 'stay_id'])
           .tail(1))

recents = recents.reset_index().drop(columns='index')

# use the most recent lab result as current data for each subject per admission
means = (pneu.groupby(['subject_id', 'hadm_id', 'stay_id'])[['heart_rate',
                                                             'sbp',
                                                             'mbp',
                                                             'resp_rate',
                                                             'temperature',
                                                             'platelet',
                                                             'wbc',
                                                             'bands',
                                                             'lactate',
                                                             'inr',
                                                             'ptt',
                                                             'creatinine',
                                                             'bilirubin',
                                                             'pneumonia']]
         .mean()
         .reset_index())
feat_squeeze = recents.combine_first(means)

In [ ]:
full_data = (subset.merge(feat_squeeze, how='left', on=['subject_id',
                                                        'hadm_id',
                                                        'stay_id'])
             .get(['heart_rate',
                   'sbp',
                   'mbp',
                   'resp_rate',
                   'temperature',
                   'platelet',
                   'wbc',
                   'bands',
                   'lactate',
                   'inr',
                   'ptt',
                   'creatinine',
                   'bilirubin',
                   'pneumonia',
                   'sepsis3']))

full_data['sepsis3'] = full_data['sepsis3'].astype(int)
full_data = full_data.rename(columns={'sepsis3': 'sepsis'}, inplace=False)

In [ ]:
def probabilistic_imputation(df, seed=42):
    """
    Impute missing values probabilistically based on the sepsis feature.
    Imputes by sampling from observed distributions.
    -----------
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe with missing values
    seed : int, optional
        Random seed for reproducibility
    --------
    Returns:
    --------
    pandas DataFrame
        The dataframe with imputed values
    """
    if seed is not None:
        np.random.seed(seed)

    # Create a copy for imputation
    imputed_df = df_cleaned = df.copy().copy()

    # Get sepsis and non-sepsis indices
    sepsis_idx = df_cleaned['sepsis'] == 1
    nonsepsis_idx = df_cleaned['sepsis'] == 0

    for col in df_cleaned.columns:
        if col == 'sepsis':
            continue  # Skip the target column

        if df_cleaned[col].isna().any():
            print(f"Imputing missing values for {col}...")
            # For binary features (like pneumonia)
            if col == 'pneumonia':
                # For sepsis patients
                sepsis_pneumonia_values = (df_cleaned.loc[sepsis_idx, col]
                                           .dropna())
                if len(sepsis_pneumonia_values) > 0:
                    # Probability of pneumonia=1
                    sepsis_pneumonia_prob = sepsis_pneumonia_values.mean()
                else:
                    sepsis_pneumonia_prob = 0.5  # Default if no data

                # For non-sepsis patients
                nonsepsis_pneumonia_values = (df_cleaned.loc[nonsepsis_idx, col]
                                              .dropna())
                if len(nonsepsis_pneumonia_values) > 0:
                    nonsepsis_pneumonia_prob = nonsepsis_pneumonia_values.mean()
                else:
                    nonsepsis_pneumonia_prob = 0.3  # Default if no data

                # Impute for sepsis patients
                missing_sepsis = sepsis_idx & df_cleaned[col].isna()
                if missing_sepsis.any():
                    n_missing = missing_sepsis.sum()
                    imputed_df.loc[missing_sepsis, col] = np.random.binomial(
                        1,
                        sepsis_pneumonia_prob,
                        size=n_missing)

                # Impute for non-sepsis patients
                missing_nonsepsis = nonsepsis_idx & df_cleaned[col].isna()
                if missing_nonsepsis.any():
                    n_missing = missing_nonsepsis.sum()
                    imputed_df.loc[missing_nonsepsis, col] = np.random.binomial(
                        1,
                        nonsepsis_pneumonia_prob,
                        size=n_missing)

            # For continuous features
            else:
                # For sepsis group - sample from observed clean values
                sepsis_values = df_cleaned.loc[sepsis_idx, col].dropna()

                # Impute for sepsis patients
                missing_sepsis = sepsis_idx & df_cleaned[col].isna()
                if missing_sepsis.any():
                    n_missing = missing_sepsis.sum()
                    imputed_df.loc[missing_sepsis, col] = np.random.choice(
                        sepsis_values,
                        size=n_missing,
                        replace=True)

                # For non-sepsis group
                nonsepsis_values = df_cleaned.loc[nonsepsis_idx, col].dropna()

                # Impute for non-sepsis patients
                missing_nonsepsis = nonsepsis_idx & df_cleaned[col].isna()
                if missing_nonsepsis.any():
                    n_missing = missing_nonsepsis.sum()
                    imputed_df.loc[missing_nonsepsis, col] = np.random.choice(
                        nonsepsis_values,
                        size=n_missing,
                        replace=True)
    print("====================================")
    print(f'Total number of imputed columns: {len(df.columns) - 1}')
    return imputed_df

In [ ]:
imputed_metadata = probabilistic_imputation(full_data, seed=42)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def compare_distributions(original_df, imputed_df, feature):
    """
    Compare distributions of original vs imputed data, separated by sepsis status
    """
    import matplotlib.pyplot as plt
    import seaborn as sns

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))

    # Sepsis patients - original data
    sns.histplot(original_df.loc[original_df['sepsis'] == 1, feature].dropna(),
                 kde=True, ax=axes[0, 0])
    axes[0, 0].set_title(f'Original {feature} - Sepsis Patients')

    # Non-sepsis patients - original data
    sns.histplot(original_df.loc[original_df['sepsis'] == 0, feature].dropna(),
                 kde=True, ax=axes[0, 1])
    axes[0, 1].set_title(f'Original {feature} - Non-sepsis Patients')

    # Sepsis patients - imputed data
    sns.histplot(imputed_df.loc[imputed_df['sepsis'] == 1, feature],
                 kde=True, ax=axes[1, 0])
    axes[1, 0].set_title(f'Imputed {feature} - Sepsis Patients')

    # Non-sepsis patients - imputed data
    sns.histplot(imputed_df.loc[imputed_df['sepsis'] == 0, feature],
                 kde=True, ax=axes[1, 1])
    axes[1, 1].set_title(f'Imputed {feature} - Non-sepsis Patients')

    plt.tight_layout()
    return fig


# Usage:
fig = compare_distributions(full_data, imputed_metadata, 'temperature')
plt.show()

In [ ]:
X = imputed_metadata.drop(columns=["sepsis"])
y = imputed_metadata.sepsis

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)


In [ ]:
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline

# CatBoost parameters
params = {
    'iterations': 3000,              # Similar to n_estimators in XGBoost
    'learning_rate': 0.4,            # Step size shrinkage
    'depth': 2,                      # Equivalent to max_depth
    'min_data_in_leaf': 4,           # Similar to min_child_weight
    'subsample': 0.8,                # Random subsampling of data
    'colsample_bylevel': 0.9,        # Similar to colsample_bytree
    'l2_leaf_reg': 0.3,              # L2 regularization
    'random_seed': 42,               # Random state for reproducibility
    'loss_function': 'Logloss',      # (equivalent to binary:logistic)
    'eval_metric': 'AUC',            # Area under curve metric
    'bootstrap_type': 'Bernoulli',   # Type of bootstrap to perform
    'verbose': False,                # Suppress verbose output
    'early_stopping_rounds':20
}

# Pipeline for classification
positive_pipeline = Pipeline([
    ('classifier', CatBoostClassifier(**params))
])

positive_pipeline.fit(X_train, y_train)
y_train_pred = positive_pipeline.predict(X_train)
y_test_pred = positive_pipeline.predict(X_test)
train_report = classification_report(y_train, y_train_pred, output_dict=True)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
print(classification_report(y_train, y_train_pred))
print(accuracy_score(y_train, y_train_pred))
print("==================", end='\n')
print(classification_report(y_test, y_test_pred))
accuracy_score(y_test, y_test_pred)

In [ ]:
train_df = pd.DataFrame(train_report).T
test_df = pd.DataFrame(test_report).T

# Add accuracy scores
train_df.loc["accuracy"] = ["", "", "", accuracy_score(y_train, y_train_pred)]
test_df.loc["accuracy"] = ["", "", "", accuracy_score(y_test, y_test_pred)]

In [ ]:
y_pred_proba = positive_pipeline.predict_proba(X_test)[:, 1]

# Compute FPR, TPR, and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color="blue", lw=2, label=f"(AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="red", linestyle="--")  # Diagonal line (random model)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")

plt.savefig("roc_curve_2.png", dpi=300, bbox_inches="tight")
plt.ylim([0.0, 1.0])

plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve
from sklearn.inspection import permutation_importance


# 1. Feature Importance
def plot_feature_importance(model, feature_names, top_n=20):
    """
    Plot feature importance from a CatBoost model
    """
    # Extract the CatBoost model from the pipeline
    catboost_model = model.named_steps['classifier']

    # Get feature importance
    importances = catboost_model.get_feature_importance()

    # Create DataFrame for plotting
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False)

    # Plot top N features
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=importance_df.head(top_n))
    plt.title('CatBoost Feature Importance')
    plt.tight_layout()
    plt.show()

    return importance_df


# 2. Permutation Importance (alternative measure that shows feature impact on performance)
def plot_permutation_importance(model, X_test, y_test, feature_names, top_n=20, n_repeats=10):
    """
    Calculate and plot permutation importance for features
    """
    # Calculate permutation importance
    perm_importance = permutation_importance(
        model, X_test, y_test, n_repeats=n_repeats, random_state=42
    )

    # Create DataFrame for plotting
    perm_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': perm_importance.importances_mean
    }).sort_values('Importance', ascending=False)

    # Plot top N features
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=perm_importance_df.head(top_n))
    plt.title('Permutation Feature Importance')
    plt.tight_layout()
    plt.show()

    return perm_importance_df


# 3. Partial Dependence Plots (shows how a feature affects predictions)
def plot_partial_dependence(model, X, feature_names, feature_idx, grid_resolution=50):
    """
    Create partial dependence plot for a specific feature
    """
    # Extract CatBoost model from pipeline
    catboost_model = model.named_steps['classifier']

    # Get feature values for the selected feature
    feature_values = np.linspace(
        np.min(X[:, feature_idx]),
        np.max(X[:, feature_idx]),
        num=grid_resolution
    )

    # Create a grid for prediction
    X_pd = np.tile(X.mean(axis=0), (grid_resolution, 1))

    # Replace the feature values with our grid
    X_pd[:, feature_idx] = feature_values

    # Make predictions
    predictions = catboost_model.predict_proba(X_pd)[:, 1]

    # Plot the partial dependence
    plt.figure(figsize=(8, 6))
    plt.plot(feature_values, predictions)
    plt.xlabel(feature_names[feature_idx])
    plt.ylabel('Predicted probability')
    plt.title(f'Partial Dependence Plot: {feature_names[feature_idx]}')
    plt.grid(True)
    plt.show()


# 4. SHAP Values (shows contribution of each feature to each prediction)
def plot_shap_values(model, X, feature_names, sample_size=100):
    """
    Calculate and plot SHAP values for CatBoost model
    """
    try:
        import shap

        # Extract CatBoost model from pipeline
        catboost_model = model.named_steps['classifier']

        # Sample data for SHAP analysis if dataset is large
        if X.shape[0] > sample_size:
            X_sample = X.sample(sample_size, random_state=42)
        else:
            X_sample = X

        # Create explainer
        explainer = shap.TreeExplainer(catboost_model)

        # Calculate SHAP values
        shap_values = explainer.shap_values(X_sample)

        # Summary plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_sample, feature_names=feature_names)

        # Dependence plots for top features
        importance = np.abs(shap_values).mean(0)
        indices = np.argsort(importance)[-5:]  # Top 5 features

        for idx in indices:
            plt.figure(figsize=(8, 6))
            shap.dependence_plot(idx, shap_values, X_sample, feature_names=feature_names)

        return explainer, shap_values

    except ImportError:
        print("SHAP package not installed. Install with: pip install shap")
        return None, None

# 5. ROC and Precision-Recall curves
def plot_model_curves(model, X_train, y_train, X_test, y_test):
    """
    Plot ROC and Precision-Recall curves for both training and test sets
    """
    # Get probabilities
    y_train_proba = model.predict_proba(X_train)[:, 1]
    y_test_proba = model.predict_proba(X_test)[:, 1]

    # Calculate ROC curve
    fpr_train, tpr_train, _ = roc_curve(y_train, y_train_proba)
    fpr_test, tpr_test, _ = roc_curve(y_test, y_test_proba)

    # Calculate AUC
    auc_train = roc_auc_score(y_train, y_train_proba)
    auc_test = roc_auc_score(y_test, y_test_proba)

    # Plot ROC curve
    plt.figure(figsize=(10, 6))
    plt.plot(fpr_train, tpr_train, label=f'Train AUC: {auc_train:.3f}')
    plt.plot(fpr_test, tpr_test, label=f'Test AUC: {auc_test:.3f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Calculate Precision-Recall curve
    precision_train, recall_train, _ = precision_recall_curve(y_train, y_train_proba)
    precision_test, recall_test, _ = precision_recall_curve(y_test, y_test_proba)

    # Plot Precision-Recall curve
    plt.figure(figsize=(10, 6))
    plt.plot(recall_train, precision_train, label='Train')
    plt.plot(recall_test, precision_test, label='Test')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

    return {
        'auc_train': auc_train,
        'auc_test': auc_test
    }


# 6. Model calibration plot
def plot_calibration_curve(model, X_train, y_train, X_test, y_test, n_bins=10):
    """
    Plot calibration curve to check if predicted probabilities match observed frequencies
    """
    # Get probabilities
    y_train_proba = model.predict_proba(X_train)[:, 1]
    y_test_proba = model.predict_proba(X_test)[:, 1]

    # Function to calculate calibration curve
    def calculate_calibration(y_true, y_prob, n_bins):
        bins = np.linspace(0, 1, n_bins + 1)
        binids = np.digitize(y_prob, bins) - 1
        bin_sums = np.bincount(binids, weights=y_prob, minlength=len(bins))
        bin_true = np.bincount(binids, weights=y_true, minlength=len(bins))
        bin_total = np.bincount(binids, minlength=len(bins))

        nonzero = bin_total != 0
        prob_true = bin_true[nonzero] / bin_total[nonzero]
        prob_pred = bin_sums[nonzero] / bin_total[nonzero]

        return prob_true, prob_pred

    # Calculate calibration for train and test
    prob_true_train, prob_pred_train = calculate_calibration(y_train, y_train_proba, n_bins)
    prob_true_test, prob_pred_test = calculate_calibration(y_test, y_test_proba, n_bins)

    # Plot calibration curve
    plt.figure(figsize=(8, 8))
    plt.plot([0, 1], [0, 1], 'k--', label='Perfectly calibrated')
    plt.plot(prob_pred_train, prob_true_train, 's-', label='Train')
    plt.plot(prob_pred_test, prob_true_test, 's-', label='Test')
    plt.xlabel('Mean predicted probability')
    plt.ylabel('Fraction of positives')
    plt.title('Calibration Curve')
    plt.legend()
    plt.grid(True)
    plt.show()


# 7. Learning curves to check for overfitting
def plot_learning_curves(model, X_train, y_train, X_test, y_test, cv=5):
    """
    Plot learning curves to check for overfitting/underfitting
    """
    from sklearn.model_selection import learning_curve

    # Calculate learning curves
    train_sizes, train_scores, test_scores = learning_curve(
        model, X_train, y_train, 
        cv=cv, 
        train_sizes=np.linspace(0.1, 1.0, 10),
        scoring='accuracy', 
        random_state=42
    )

    # Calculate mean and std
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Plot learning curves
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, label='Training score')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
    plt.plot(train_sizes, test_mean, label='Cross-validation score')
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
    plt.xlabel('Training examples')
    plt.ylabel('Accuracy')
    plt.title('Learning Curves')
    plt.legend()
    plt.grid(True)
    plt.show()


def run_full_analysis(model, X_train, y_train, X_test, y_test, feature_names):
    """
    Run a comprehensive model interpretation and analysis
    """
    print("=== Model Analysis ===")
    print("1. Feature Importance")
    importance_df = plot_feature_importance(model, feature_names)
    print(importance_df.head(10))

    print("\n2. Permutation Importance")
    perm_importance_df = plot_permutation_importance(model, X_test, y_test, feature_names)
    print(perm_importance_df.head(10))

    print("\n3. Model Performance Curves")
    metrics = plot_model_curves(model, X_train, y_train, X_test, y_test)

    print("\n4. Model Calibration")
    plot_calibration_curve(model, X_train, y_train, X_test, y_test)

    print("\n5. Learning Curves")
    plot_learning_curves(model, X_train, y_train, X_test, y_test)

    print("\n6. Partial Dependence Plots for top features")
    for idx in importance_df.head(5).index:
        feature_idx = list(feature_names).index(importance_df.iloc[idx]['Feature'])
        plot_partial_dependence(model, X_train.values, feature_names, feature_idx)

    print("\n7. SHAP Analysis")
    plot_shap_values(model, X_train, feature_names)

    return {
        'feature_importance': importance_df,
        'permutation_importance': perm_importance_df,
        'performance_metrics': metrics
    }

# Example of usage:
run_full_analysis(positive_pipeline, X_train, y_train, X_test, y_test, X_train.columns)

In [ ]:
plot_shap_values(positive_pipeline, X_train, X_train.columns)

In [ ]:
import pickle

# Save the model to a file
with open("best_second_model.pkl", "wb") as file:
    pickle.dump(positive_pipeline, file)